In [ ]:
import pandas as pd
import shorthand as shnd

s = shnd.Shorthand(
    entry_syntax="shorthand/resources/default_entry_syntax.csv",
    link_syntax="shorthand/resources/default_link_syntax.csv",
    syntax_case_sensitive=False
)
'''
parsed = s.parse_text(
    'shorthand/test_data/manual_annotation.shnd',
    skiprows=2,
    comment_char='#'
)
'''

parsed = s.parse_text(
    'shorthand/test_data/single_column.shnd',
    item_separator='__',
    default_entry_prefix='wrk',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    skiprows=0,
    comment_char='#',
    drop_na=False
)

links = parsed.links
links.iloc[60:]
'''
print('       strings', parsed.strings.memory_usage(deep=True).sum()/1000, 'kb')
print('         links', parsed.links.memory_usage(deep=True).sum()/1000, 'kb')
print('resolved links', parsed.resolve_links().memory_usage(deep=True).sum()/1000, 'kb')
'''
entry_type = parsed.id_lookup('link_types', 'entry')
entry_string_ids = parsed.links.loc[parsed.links['link_type_id'] == entry_type, 'tgt_string_id']

parsed.resolve_links().query('src_string.str.contains("nasa")').query('src_node_type != "shorthand_text"').query('tgt_node_type != "shorthand_text"')

parsed.resolve_links().query('link_type == "cited"').merge(parsed.links, left_index=True, right_index=True)

s = parsed.synthesize_shorthand_entries('wrk', fill_spaces=True)

check = pd.Series([
    'asmith_bwu__1999__s_bams__101__803__xxx',
    'asmith_bwu__1998__s_bams__100__42__yyy',
    'bjones__1975__s_jats__90__1__!',
    'bwu__1989__t_long|title__!__80__!',
    'Some|Author__1989__t_A|Title|With|\\#__!__!__!',
    'asmith_bwu__2008__s_bams__110__1__zzz'
])

(check == s).all()

In [ ]:
import pandas as pd
import shorthand as shnd
from bibtexparser.bparser import BibTexParser

bibtex_parser = BibTexParser(common_strings=True)
with open("shorthand/test_data/bibtex_test_data_short.bib", encoding='utf8') as f:
    bibdatabase = bibtex_parser.parse_file(f)

data = pd.DataFrame(bibdatabase.entries)

s = shnd.Shorthand(
    entry_syntax="shorthand/resources/default_bibtex_syntax.csv",
    syntax_case_sensitive=False
)

parsed = s.parse_items(
    data,
    space_char='|',
    na_string_values='!',
    na_node_type='missing'
)

parsed_identifiers = parsed.strings.query('node_type_id == 8')

check = pd.Series([
    '10.1038/194638b0',
    '10.1175/1520-0493(1962)090<0311:OTOKEB>2.0.CO;2',
    '10.3402/tellusa.v14i3.9551',
    '10.1175/1520-0477-43.9.451',
    '10.3402/tellusa.v14i4.9569',
    '10.1007/BF02317953',
    '10.1007/BF02247180',
    '10.1029/JZ068i011p03345',
    '10.1029/JZ068i009p02375',
])

assert (check == parsed_identifiers['string'].array).all()

(check == parsed_identifiers['string'].array).all()

In [ ]:
from bibtexparser.bibdatabase import BibDatabase as _bibtex_db
from bibtexparser import dumps as _dump_bibtex_string

def btwriter(entry_series):
    db = _bibtex_db()
    db.entries = [dict(entry_series.dropna().map(str))]
    return _dump_bibtex_string(db)

print(data.apply(btwriter, axis=1)[9])

In [1]:
import shorthand as shnd

s = shnd.Shorthand(
    entry_syntax="shorthand/resources/default_entry_syntax.csv",
    link_syntax="shorthand/resources/default_link_syntax.csv",
    syntax_case_sensitive=False
)

parsed = s.parse_text(
    #'shorthand/test_data/manual_annotation.shnd',
    'shorthand/test_data/manual_annotation_with_self_descriptive_entries.shnd',
    #'shorthand/test_data/self_descriptive_source_data.shnd',
    item_separator='__',
    default_entry_prefix='wrk',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    skiprows=2,
    comment_char='#'
)

!pytest shorthand/tests.py

#print(parsed.link_tags.merge(parsed.strings, left_on='tag_string_id', right_index=True))
#parsed.resolve_links().query('(tgt_node_type == "sdNode2") | (src_node_type == "sdNode2")')
#parsed.resolve_links().query('(tgt_node_type == "work") & (src_node_type == "work")')
#parsed.resolve_links().query('link_type == "funder"')
#parsed.resolve_links().iloc[:40]

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0
rootdir: c:\Users\short\Dropbox\jd\60-69 projects\61 public\61.03 shorthand
plugins: anyio-3.4.0
collected 12 items

shorthand\tests.py ............                                          [100%]

============================= 12 passed in 3.66s ==============================


In [ ]:
parsed.resolve_links().iloc[:40]

In [ ]:
parsed.resolve_links().iloc[40:]

In [ ]:
parsed.resolve_strings().query('string.str.contains("J")')

In [ ]:
import pandas as pd

item_separator = '__'

prefixes=['wrk', 'ack', item_separator]
prefixes_regex = '|'.join(prefixes)
prefixes_regex = '^({})(?={})'.format(prefixes_regex, item_separator)

entries = pd.Series([
    'wrk__the rest of the entry',
    'no prefix',
    '____starts with separator',
    'ack__another entry',
    '____NewPrefix____has missing value',
    
])

entries.str.split(prefixes_regex, n=1, regex=True, expand=True).replace('', pd.NA).fillna(pd.NA)

In [ ]:
import pandas as pd

pd.Series([1]).any(axis=None)


In [4]:
    import pandas as pd
    import shorthand as shnd
    from io import StringIO

    s = shnd.Shorthand(
        entry_syntax="shorthand/resources/default_entry_syntax.csv",
        link_syntax="shorthand/resources/default_link_syntax.csv",
        syntax_case_sensitive=False
    )

    shorthand = (
        'left_entry, right_entry, link_tags_or_override, reference\n'
        'Auth1__2000__BAMS__100__!__xxx__ tag1 tag2, '
        'Auth2__1990__BAMS__90__40__yyy__ tag1 tag3, '
        'LinkTag'
    )

    self_descriptive = (
        'left_entry, right_entry, link_tags_or_override, reference\n'
        '____work__author_actor_Auth1__published_date_2000__supertitle_work_BAMS__volume_work_100__page_work_!__doi_identifier_xxx__ tag1 tag2, '
        '____work__author_actor_Auth2__published_date_1990__supertitle_work_BAMS__volume_work_90__page_work_40__doi_identifier_yyy__ tag1 tag3, '
        'lt__cited LinkTag'
    )

    parsed_shnd_sources = s.parse_text(
        StringIO(shorthand),
        item_separator='__',
        default_entry_prefix='wrk',
        space_char='|',
        na_string_values='!',
        na_node_type='missing',
        comment_char='#'
    )

    parsed_s_d_sources = s.parse_text(
        StringIO(self_descriptive),
        item_separator='__',
        default_entry_prefix='wrk',
        space_char='|',
        na_string_values='!',
        na_node_type='missing',
        comment_char='#'
    )

    shnd_sources_link_types = parsed_shnd_sources.resolve_links()[
        'link_type'
    ]
    s_d_sources_link_types = parsed_s_d_sources.resolve_links()[
        'link_type'
    ]
    (shnd_sources_link_types.sort_values() == s_d_sources_link_types.sort_values()).all()

ValueError: Can only compare identically-labeled Series objects

In [22]:
vals1 = parsed_shnd_sources.resolve_strings()
vals1 = vals1.loc[~vals1['node_type'].isin(['entry', 'shorthand_text', 'shorthand_link_syntax', 'shorthand_entry_syntax', 'work'])]
vals1 = vals1.loc[vals1['string'] != '1']
vals1 = vals1.sort_values(by='string').reset_index(drop=True)
vals2 = parsed_s_d_sources.resolve_strings()
vals2 = vals2.loc[~vals2['node_type'].isin(['entry', 'shorthand_text', 'shorthand_link_syntax', 'shorthand_entry_syntax', 'work'])]
vals2 = vals2.sort_values(by='string').reset_index(drop=True)

vals1 == vals2

,string,node_type
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,True,True
7,True,True
8,True,True
9,True,True


In [13]:
vals = parsed_s_d_sources.resolve_strings()
vals.loc[~vals['node_type'].isin(['entry', 'shorthand_text', 'shorthand_link_syntax', 'shorthand_entry_syntax']), 'string']

0                                                 Auth1
1                                                  2000
2                                                  BAMS
3                                                   100
4                                                     !
5                                                   xxx
6     ____work__author_actor_Auth1__published_date_2...
7                                                 Auth2
8                                                  1990
9                                                    90
10                                                   40
11                                                  yyy
12    ____work__author_actor_Auth2__published_date_1...
13                                                 tag1
14                                                 tag2
15                                                 tag3
16                                              LinkTag
18                                 Shorthand.par